In [1]:
with open('input.txt', 'r', encoding = 'utf-8') as f:
    text = f.read()

In [2]:
print('Length of the dataset in characters: ', len(text))

Length of the dataset in characters:  1115394


In [3]:
print(text[:100])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You


In [4]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(len(chars))


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [5]:
#create a mapping from characters to integers
stoi = {ch:i for i, ch in enumerate(chars)}
itos = {i:ch for i, ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s] #take a string and return a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) #take a list of characters and return a string

print(encode("hello world"))
print(decode(encode("hello world")))

[46, 43, 50, 50, 53, 1, 61, 53, 56, 50, 42]
hello world


In [6]:
import torch
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:100])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


In [7]:
n = int(len(data) * 0.9)
train_data = data[:n]
val_data = data[n:]

In [8]:
context_length = 8
train_data[:context_length + 1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [9]:
x = train_data[:context_length]
y = train_data[1:context_length + 1]
for t in range(context_length):
    context = x[:t+1]
    target = y[t]
    print(f'When the input is {context}, the output is {target}')

When the input is tensor([18]), the output is 47
When the input is tensor([18, 47]), the output is 56
When the input is tensor([18, 47, 56]), the output is 57
When the input is tensor([18, 47, 56, 57]), the output is 58
When the input is tensor([18, 47, 56, 57, 58]), the output is 1
When the input is tensor([18, 47, 56, 57, 58,  1]), the output is 15
When the input is tensor([18, 47, 56, 57, 58,  1, 15]), the output is 47
When the input is tensor([18, 47, 56, 57, 58,  1, 15, 47]), the output is 58


In [12]:
torch.manual_seed(42)
batch_size = 4 #how many sequences we want to process in parallel
context_length = 8

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - context_length, (batch_size,))
    x = torch.stack([data[i:i+context_length] for i in ix])
    y = torch.stack([data[i+1:i+context_length+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print('inputs: ')
print(xb.shape, xb)
print('outputs: ')
print(yb.shape, yb)

inputs: 
torch.Size([4, 8]) tensor([[57,  1, 46, 47, 57,  1, 50, 53],
        [ 1, 58, 46, 43, 56, 43,  1, 41],
        [17, 26, 15, 17, 10,  0, 32, 53],
        [57, 58,  6,  1, 61, 47, 58, 46]])
outputs: 
torch.Size([4, 8]) tensor([[ 1, 46, 47, 57,  1, 50, 53, 60],
        [58, 46, 43, 56, 43,  1, 41, 39],
        [26, 15, 17, 10,  0, 32, 53,  1],
        [58,  6,  1, 61, 47, 58, 46,  0]])


In [14]:
import torch.nn as nn
from torch.nn import functional as F

class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        #each token reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets = None):
        #idx and targets are both (B,T) tensors of integers
        logits = self.token_embedding_table(idx) #(B,T,C)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        return logits, loss
    
    def generate(self,idx,max_new_tokens):
        #idx is a (B,T) array of indices in the current context
        for _ in range(max_new_tokens):
            #get the predictions
            logits, loss = self(idx)
            #focus only on the last time step
            logits = logits[:, -1, :] #becomes (B,C)
            probs = F.softmax(logits, dim = -1) #(B,C)
            #sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) #(B,1)
            #append to the running sequence
            idx = torch.cat((idx, idx_next), dim = 1)
        return idx
    
m = BigramLanguageModel(vocab_size)
logits,loss=m(xb, yb)
print(logits.shape)
print(loss)

print(decode(m.generate(idx = torch.zeros((1,1), dtype = torch.long), max_new_tokens=100)[0].tolist()))

torch.Size([32, 65])
tensor(4.7241, grad_fn=<NllLossBackward0>)

cfYCDRUZsYBsA?Y?vgB!ZWOEiAoezL:q&Avufr?gSGdWrp&Bxt-R?wo'TYhBChdIC-RDaRmEGENyouVg'UjyQNyQSpZUVeN:BZqh


In [15]:
optimizer = torch.optim.AdamW(m.parameters(), lr = 1e-3)
batch_size = 32

In [17]:
for steps in range(30000):
    xb, yb = get_batch('train')

    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())

2.3734991550445557


In [19]:
print(decode(m.generate(idx = torch.zeros((1,1), dtype = torch.long), max_new_tokens=500)[0].tolist()))


ANGRD forou t Yoorr chal tris onice it siasen't bein stche crcelldiond, g frer knde, nersoy st nd cowidolewil fro toeard, thy the'soze tar?
ABumy, veschiethe tosprve.
catobe hourent itadict se
Theawan wing we atowir u am'seme Wayow wh thollds' w sd! tonge hersoxe benorst po han y sisd eankil hokils it,
AROMayvird mancy ls
Fimay d s f t ciniressull gtournooksinor'd prurs

Fidell nt me y ofin go-h NGHarvedes ngrser hion fo thor jubughars
A mim,

ARI anyoumiounoomer mit tofayod,
Trd mels
CLIIng ce,


In [23]:
B,T,C = 4,8,32
x = torch.randn(B,T,C)

head_size = 16
key = nn.Linear(C, head_size, bias = False)
query = nn.Linear(C, head_size, bias = False)
value = nn.Linear(C, head_size, bias = False)
k = key(x) #(B,T,head_size)
q = query(x) #(B,T,head_size)
wei = q @ k.transpose(-2, -1) #(B,T,head_size) @ (B, head_size, T) --> (B,T,T)

tril = torch.tril(torch.ones(T,T))
wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim = -1)
v = value(x)
out = wei @ v

out.shape

torch.Size([4, 8, 16])

In [22]:
wei

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])

In [24]:
n_embd = 32

class Head:
    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(context_length, context_length)))

    def forward(self,x):
        B,T,C = x.shape
        k = self.key(x)
        q = self.query(x)
        #compute affinities
        wei = q @ k.transpose(-2,-1) * C**-0.5 #(B,T,C) @ (B, C, T) --> (B,T,T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) #(B,T,T)
        wei = F.softmax(wei, dim = -1)
        #perform weighted aggregation
        v = self.value(x) #(B,T,C)
        out = wei @ v #(B,T,T) @ (B,T,C) --> (B,T,C)
        return out
    

class MultiHeadAttention(nn.Module):
    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])

    def forward(self, x):
        return torch.cat([h(x) for h in self.heads], dim = -1)
    
class FeedForward(nn.Module):
    def __init__(self, n_embd) -> None:
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd,n_embd),
            nn.ReLU(),
        )

    def forward(self, x):
        return self.net(x)



We've changed the Bigram model enough that we should redefine it - it's a small GPT model now

In [26]:
import torch
import torch.nn as nn
from torch.nn import functional as F

# hyperparameters
batch_size = 32 # how many independent sequences will we process in parallel?
context_length = 64 # what is the maximum context length for predictions?
max_iters = 5000
eval_interval = 500
learning_rate = 3e-4
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 64 #384/6 = 64-dimensional head
n_head = 4
n_layer = 2 #6 layers of the above
dropout = 0.2 #prevents overfitting

torch.manual_seed(42)

with open('input.txt', 'r', encoding = 'utf-8') as f:
    text = f.read()


chars = sorted(list(set(text)))
vocab_size = len(chars)
#create a mapping from characters to integers
stoi = {ch:i for i, ch in enumerate(chars)}
itos = {i:ch for i, ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s] #take a string and return a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) #take a list of characters and return a string
data = torch.tensor(encode(text), dtype=torch.long)
n = int(len(data) * 0.9)
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - context_length, (batch_size,))
    x = torch.stack([data[i:i+context_length] for i in ix])
    y = torch.stack([data[i+1:i+context_length+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X,Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

class Head(nn.Module):
    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(context_length, context_length)))
        self.dropout = nn.Dropout(dropout) #after we calculate affinities

    def forward(self,x):
        B,T,C = x.shape
        k = self.key(x)
        q = self.query(x)
        #compute affinities
        wei = q @ k.transpose(-2,-1) * C**-0.5 #(B,T,C) @ (B, C, T) --> (B,T,T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) #(B,T,T)
        wei = F.softmax(wei, dim = -1)
        wei = self.dropout(wei)
        #perform weighted aggregation
        v = self.value(x) #(B,T,C)
        out = wei @ v #(B,T,T) @ (B,T,C) --> (B,T,C)
        return out

class MultiHeadAttention(nn.Module):
    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim = -1)
        out = self.dropout(self.proj(out))
        return out
    
class FeedForward(nn.Module):
    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd,n_embd), #projection layer for residual output
            nn.Dropout(dropout), #right before the residual connection back into the pathway
        )

    def forward(self, x):
        return self.net(x)
    
class Block(nn.Module):
    def __init__(self, n_embd, n_head):
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedForward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        #communication and computation, along with residual connections
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x


class GPT(nn.Module):
    def __init__(self):
        super().__init__()
        #each token reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embeddings_table = nn.Embedding(context_length, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd)
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets = None):
        B, T = idx.shape

        #idx and targets are both (B,T) tensors of integers
        token_emb = self.token_embedding_table(idx) #(B,T,C)
        pos_emb = self.position_embeddings_table(torch.arange(T, device = device)) #(T,C)
        x = token_emb + pos_emb
        x = self.blocks(x) #(B,T,C)
        x = self.ln_f(x)
        logits = self.lm_head(x) #(B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        return logits, loss
    
    def generate(self,idx,max_new_tokens):
        #idx is a (B,T) array of indices in the current context
        for _ in range(max_new_tokens):
            #crop idx to the last context_length tokens
            idx_cond = idx[:, -context_length:]
            #get the predictions
            logits, loss = self(idx_cond)
            #focus only on the last time step
            logits = logits[:, -1, :] #becomes (B,C)
            probs = F.softmax(logits, dim = -1) #(B,C)
            #sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) #(B,1)
            #append to the running sequence
            idx = torch.cat((idx, idx_next), dim = 1)
        return idx
    
model = GPT()
m = model.to(device)

optimizer = torch.optim.AdamW(m.parameters(), lr = 1e-3)

for iter in range(max_iters):
    if iter % eval_interval == 0:
        losses = estimate_loss()
        print(f'step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}')

    xb, yb = get_batch('train')

    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

context = torch.zeros((1,1), dtype = torch.long, device = device)
print(decode(m.generate(context, max_new_tokens=500)[0].tolist()))

step 0: train loss 4.4074, val loss 4.4080
step 500: train loss 2.3712, val loss 2.3803
step 1000: train loss 2.1458, val loss 2.1782
step 1500: train loss 2.0257, val loss 2.0743
step 2000: train loss 1.9443, val loss 2.0213
step 2500: train loss 1.8848, val loss 1.9746
step 3000: train loss 1.8357, val loss 1.9440
step 3500: train loss 1.8044, val loss 1.9262
step 4000: train loss 1.7802, val loss 1.9088
step 4500: train loss 1.7611, val loss 1.8912

To it uswarstern fould thread not yout mape,
Alt stilen And agaip noso.

BUCKI:
I thrors of they lose of speto phoce;
Af crowarleo, the has youn hankfige law tremines me
aon to sever so it Ghours love.

HAST; a goving most:
That's I
Marper truch.

SIA:
I tho will:
I we ourah, lard thing like! why sweep,
De Ame age my.

HOFRIZEL:
A Ver:
But 
Hegiend.

GLOUKE
Thathen I st witleep I it boot an Pity not, To to year
Saurs mist thoney therer of lintrey yoursess;
But died it cloing all:
Montle whit sha


As we can see, we're getting text that is a little more comprehensible than before. This is down to the fact that our Model is now much deeper. But the actual depth required to make this a respectable decoder-only pre-trained transformer is a little more, and we can't train that on a CPU. Those hyperparams are provided in the accompanying gpt.py